<a href="https://colab.research.google.com/github/roscoekerby/FREE-OpenCV-Bootcamp/blob/main/Copy_of_12_Face_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from IPython.display import display, Javascript, HTML
from google.colab.output import eval_js
from base64 import b64decode

def record_video(filename='video.mp4'):
    js = Javascript('''
    async function recordVideo() {
      const startButton = document.createElement('button');
      const stopButton = document.createElement('button');
      startButton.textContent = 'Start Recording';
      stopButton.textContent = 'Stop Recording';
      document.body.appendChild(startButton);
      document.body.appendChild(stopButton);
      stopButton.style.display = 'none';

      const video = document.createElement('video');
      const stream = await navigator.mediaDevices.getUserMedia({ video: true });
      video.srcObject = stream;
      video.play();

      let recorder;
      let chunks = [];

      startButton.onclick = () => {
        recorder = new MediaRecorder(stream);
        recorder.ondataavailable = e => chunks.push(e.data);
        recorder.start();
        startButton.style.display = 'none';
        stopButton.style.display = 'inline';
      };

      stopButton.onclick = async () => {
        recorder.stop();
        stopButton.style.display = 'none';

        await new Promise(resolve => recorder.onstop = resolve);
        const blob = new Blob(chunks, { type: 'video/mp4' });
        const url = URL.createObjectURL(blob);
        const a = document.createElement('a');
        a.style.display = 'none';
        a.href = url;
        a.download = 'video.mp4';
        document.body.appendChild(a);
        a.click();
        URL.revokeObjectURL(url);
        stream.getTracks().forEach(track => track.stop());
      };
    }
    recordVideo();
    ''')
    display(js)

# Record a video with manual start and stop
record_video()


<IPython.core.display.Javascript object>

In [18]:
from google.colab import files
uploaded = files.upload()

# Assume the video file is uploaded as 'video.mp4'
video_path = 'video.mp4'


Saving video (1).mp4 to video (1).mp4


In [20]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# Load the face detection model
net = cv2.dnn.readNetFromCaffe("deploy.prototxt", "res10_300x300_ssd_iter_140000_fp16.caffemodel")
in_width = 300
in_height = 300
mean = [104, 117, 123]
conf_threshold = 0.7

# Open the uploaded video file
video_path = '/content/video (1).mp4'
cap = cv2.VideoCapture(video_path)

# Get the width and height of the video frames
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create a VideoWriter object
output_path = 'output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for mp4
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_height = frame.shape[0]
    frame_width = frame.shape[1]

    # Create a 4D blob from the frame
    blob = cv2.dnn.blobFromImage(frame, 1.0, (in_width, in_height), mean, swapRB=False, crop=False)
    net.setInput(blob)
    detections = net.forward()

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x_left_bottom = int(detections[0, 0, i, 3] * frame_width)
            y_left_bottom = int(detections[0, 0, i, 4] * frame_height)
            x_right_top = int(detections[0, 0, i, 5] * frame_width)
            y_right_top = int(detections[0, 0, i, 6] * frame_height)

            cv2.rectangle(frame, (x_left_bottom, y_left_bottom), (x_right_top, y_right_top), (0, 255, 0))
            label = "Confidence: %.4f" % confidence
            label_size, base_line = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)

            cv2.rectangle(
                frame,
                (x_left_bottom, y_left_bottom - label_size[1]),
                (x_left_bottom + label_size[0], y_left_bottom + base_line),
                (255, 255, 255),
                cv2.FILLED,
            )
            cv2.putText(frame, label, (x_left_bottom, y_left_bottom), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))

    t, _ = net.getPerfProfile()
    label = "Inference time: %.2f ms" % (t * 1000.0 / cv2.getTickFrequency())
    cv2.putText(frame, label, (0, 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0))

    # Write the frame to the output video
    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()

In [21]:
# Installing ffmpeg
!apt-get -qq install ffmpeg

# Change video encoding of mp4 file from XVID to h264
!ffmpeg -y -i {output_path} -c:v libx264 $"encoded_vid.mp4"  -hide_banner -loglevel error

In [22]:
from IPython.display import HTML
from base64 import b64encode

def display_video(file_path):
    with open(file_path, "rb") as f:
        video_data = f.read()
    video_base64 = b64encode(video_data).decode()
    video_html = f'''
    <video width="600" height="400" controls>
        <source src="data:video/mp4;base64,{video_base64}" type="video/mp4">
    </video>
    '''
    return HTML(video_html)

# Example usage
display_video('/content/encoded_vid.mp4')
